In [1]:
import plotly as py
import plotly.graph_objs as go
import random as rand
import numpy as np
import math
import ipywidgets as widgets

py.offline.init_notebook_mode(connected=True)

In [2]:
# camel problem -- global minumum -1.0316 -- optimization variables 2
def fitnessP1(x):
    global rangesOfOptVars
    rangesOfOptVars = [[-3,3],[-2,2]]
    a = (4-2.1*(x[0]**2)+(x[0]**4)/3)*(x[0]**2)+x[0]*x[1]+(-4 + 4*(x[1]**2))*(x[1]**2)
    return a

# Branin problem -- global minumum 0.397887 -- optimization variables 2
def fitnessP2(x):
    global rangesOfOptVars
    # [ [min-x1, max-x1], [min-x2, max-x2] ]
    rangesOfOptVars = [[-5,10],[0,15]]
    a = (x[1]-(5.1/(4*(math.pi)**2))*(x[0]**2)+(5/math.pi)*x[0])**2+10*(1-1/(8*math.pi))*math.cos(x[0])+10
    return a

# Griewank problem -- global minumum 0.0 -- optimization variables i (1,2)
def fitnessP3(x):
    global rangesOfOptVars
    rangesOfOptVars = [[-100,100] for i in range(noOfOptVars)]
    sumofsquares, multipleofcosines = 0, 1
    for i in range(noOfOptVars):
        sumofsquares = sumofsquares + x[i]**2
        multipleofcosines = multipleofcosines * math.cos(x[i]/math.sqrt(i+1))
    a = 1 + (1/200)*sumofsquares-multipleofcosines
    return a

# Cosine mixture problem -- global minumum -0.4 -- optimization variables 4
def fitnessP4(x):
    global rangesOfOptVars
    rangesOfOptVars = [[-1,1] for i in range(noOfOptVars)]
    sumofsquares, sumofcosines = 0, 0
    for i in range(noOfOptVars):
        sumofsquares = sumofsquares + x[i]**2
        sumofcosines = sumofcosines + math.cos(5*math.pi*x[i])
    a = sumofsquares - 0.1*sumofcosines
    return a
# function switcher
switcher = {
    0: fitnessP1,
    1: fitnessP2,
    2: fitnessP3,
    3: fitnessP4
}
# function's optimization variable number
switcherOfVarNo = {
    0: 2,
    1: 2,
    2: 1,
    3: 4
}
# global variable for functions' optimization variables number
noOfOptVars = 0
# global variable for functions' optimization variables ranges (search space)
rangesOfOptVars = [[0]*noOfOptVars for s in switcher]

# main fitness functions which does the switch according to function index in "switcher"
def fitness(funcIndex,x):
    func = switcher.get(funcIndex, lambda x: "Invalid index") # Get the function from switcher dictionary
    global noOfOptVars
    noOfOptVars = switcherOfVarNo.get(funcIndex)
    return func(x) # Execute the function

In [3]:
iterations = 1000
seedsArr = [] # 5 different seed set used for every problem, 6 seeds per set
# seeds [N, MUs, MUk, Delta-t, alpha, beta, resultsForSeedSet]
seedsArr.append([15, 1, 0.9, 1, 0.9, 0.05, []])
seedsArr.append([10, 1, 0.5, 1, 0.99, 0.1, []])
seedsArr.append([12, 1, 0.8, 1, 0.95, 0.05, []])
seedsArr.append([20, 1, 0.9, 1, 0.92, 0.05, []])
seedsArr.append([10, 1, 0.1, 1, 0.9, 0.5, []])

In [4]:
results = [] # diffrent problems' diffret seed results
for r in range(len(switcher)):
    # setting the r's problem function, variable number and variable ranges as the defualt
    fitness(r,[0]*switcherOfVarNo.get(r)) 
    problemResult = [] # same problem diffrent seeds results
    for seeds in seedsArr: # iterate on seeds
        N = seeds[0] # population of candidate solutions (teams)
        n = noOfOptVars # set by the global variable according to the function r
        seeds[6] = [] # results for "r" problem, "seeds" set
        
        # -- step 1 -- Initialization
        # teams (candidate solutions ) multi dimension array [[xi1, xi2, .., xij]]
        teams = [[] for x in range(n)]
        for i in range(N):
            # every i'th team
            for x in range(n):
                # every i'th team's jth variable is generated according to crossponding variable range
                teams[x].append(rand.uniform(rangesOfOptVars[x][0],rangesOfOptVars[x][1]))
        
        # fitness array
        fit = [0]*N
        # weight array
        w = [0]*N
        
        # initializing seeds (constants)
        Ms = seeds[1]
        Mk = seeds[2]
        dt = seeds[3]
        alpha = seeds[4] # [0.9 , 0.99]
        beta = seeds[5] # (0,1]
        
        # -- step 6 -- Termination: repeate until a termination criterion is satisfied
        k = 1
        while k <= iterations:
            # -- step 2 -- Evaluation of candidate designs and weight assignment
            # evaluting every team
            for i in range(N):
                # calling r'th fitness function with crossponding x variables array
                fit[i] = (fitness(r,[x[i] for x in teams]))    
                
            # sorting fitness array in paralel with teams array
            # the best (minumum) solution is at index 0 
            fit, teams  = (list(t) for t in zip(*sorted(zip(fit, [*zip(*teams)]))))
            teams = [list(t) for t in [*zip(*teams)]]
            
            # weights of the teams are calculated - eq(5)
            for i in range(N):
                diff = fit[0]-fit[-1]
                if(diff!=0):
                    w[i]= (fit[i]-fit[-1])/(fit[0]-fit[-1]) + 1
            
            # -- step 3 -- Competition and displacement
            # each i team competes with other (j) teams
            for i in range(N):
                # team i total displacement for all optimization variables (xi)
                dxi = [0]*n 
                for j in range(N):
                    # pulling force between teams i and j is determined
                    fpij = max(w[i]*Ms,w[j]*Ms)
                    # only when Fp is bigger team i moves
                    if fpij > w[i]*Ms:
                        # force affecting team i from team j - eq(6)
                        frij = fpij - w[i]*Mk
                        # g,a and delta-x for every team's xij (optimization variable) are calculated
                        for x in range(n):
                            gxij = teams[x][j] - teams[x][i] # eq(8)
                            axij = (frij/(w[i]*Mk))*gxij # eq(7)
                            dxij = 0.5*axij*(dt**2) + alpha*beta*(rangesOfOptVars[x][1]-rangesOfOptVars[x][0])*rand.gauss(0,1) # eq(9)
                            dxi[x]=dxi[x]+dxij # eq(10)
                            
                # for all i'th team's xij adding the displacement 
                for x in range(n):
                    teams[x][i] = teams[x][i]+dxi[x] # eq(11)
                    # -- step 5 -- Handling the side constraints
                    if(teams[x][i]<rangesOfOptVars[x][0] or teams[x][i]>rangesOfOptVars[x][1]):
                        teams[x][i] = teams[x][0] + (rand.gauss(0,1)/k)*(teams[x][0] - teams[x][i]) # eq(12)
                # -- step 4 -- Updating the league
                newTeamFit = fitness(r,[x[i] for x in teams])
                if(newTeamFit<fit[-1]):
                    fit[-1] = newTeamFit
            # decreasing alpha
            alpha = alpha-alpha*k/iterations
            k+=1
            # adding best fitness value to this seeds result set
            seeds[6].append(min(fit))
        # adding seed set results to r function result set
        problemResult.append(seeds[6])
    # adding r problems results set to total results
    results.append(problemResult)


In [5]:
x = np.linspace(0,iterations,iterations)
layout = go.Layout(
    title='TUG OF WAR OPTIMIZATION',
    yaxis=dict(
        title='fitness'
    ),
    xaxis=dict(
        title='iteration'
    )
)

def update_plot(graphs,graphChildren):
    data = []
    for i in graphs:
        for j in graphChildren:
            trace1 = go.Scatter(
                x=x,
                y=results[i-1][j-1],
                mode='lines',
                name='prob{} seed{}'.format(i,j),
                line=dict(
                    shape='spline'
                )
            )
            data.append(trace1)
    fig = go.Figure(data=data, layout=layout)
    py.offline.iplot(fig)

graphs = widgets.SelectMultiple(options=list(range(1,len(results)+1)), value=(1, ),description='Problem Index')
graphChildren = widgets.SelectMultiple(options=list(range(1,len(results[0])+1)), value=(1, ),description='Seed Index')

widgets.interactive(update_plot,graphs=graphs, graphChildren=graphChildren)


interactive(children=(SelectMultiple(description='Problem Index', index=(0,), options=(1, 2, 3, 4), value=(1,)), SelectMultiple(description='Seed Index', index=(0,), options=(1, 2, 3, 4, 5), value=(1,)), Output()), _dom_classes=('widget-interact',))